In [1]:
import numpy as np
from keras import backend as K
from keras.engine.topology import Layer
from keras.layers import Input,Dense,Reshape,UpSampling2D,Conv2DTranspose
from keras.models import Model
from src.models import ResBlock

Using TensorFlow backend.


In [63]:
input_shape=(20,)
dim = int(56/4)
ch = 16
dec_channels = [2,1]
inputs = Input(shape=input_shape)
class_cond = Input(shape=(12,))

init_shape = (dim,dim,dec_channels[0]*ch)
x = Dense(np.prod(init_shape))(inputs)
x = Reshape(init_shape)(x)
x = FiLM(class_cond,init_shape)(x)

# x = ResBlock(1*ch,(3,3),block_id=1)(x)


# x = UpSampling2D()(x)
# outputs = Conv2DTranspose(1,kernel_size=(3,3),padding='same')(x)

# mod = Model(inputs,outputs)

(None, 14, 14, 32)


In [57]:
x = fl(x)


(None, 14, 14, 32)
(None, 32)
(None, 32)


In [60]:
K.int_shape(x*fl.gamma)

(None, 14, 14, 32)

In [ ]:
fl.compute_output_shape(init_shape)

In [4]:
K.int_shape(x)

(None, 56, 56, 16)

In [62]:
class FiLM(Layer):
    def __init__(self, conditioner, output_dim, **kwargs):
        self.output_dim = output_dim
        self.conditioner = conditioner
        self.conditioner_shape = K.int_shape(self.conditioner)
        super(FiLM, self).__init__(**kwargs)

    def build(self, input_shape):
        print(input_shape)
        # Create a trainable weight variable for this layer.
        self.gamma_k = self.add_weight(name='gamma_kernel', 
                                      shape=(self.conditioner_shape[-1], self.output_dim[-1],),
                                      initializer='glorot_normal',
                                      trainable=True)
        
        self.gamma_b = self.add_weight(name='gamma_bias', 
                                      shape=(self.output_dim[-1],),
                                      initializer='glorot_normal',
                                      trainable=True)
        
                

        
        self.beta_k = self.add_weight(name='beta_kernel', 
                                      shape=(self.conditioner_shape[-1],self.output_dim[-1]),
                                      initializer='glorot_normal',
                                      trainable=True)
        self.beta_b = self.add_weight(name='beta_bias', 
                                      shape=(self.output_dim[-1],),
                                      initializer='glorot_normal',
                                      trainable=True)
        
        self.gamma = K.dot(self.conditioner,self.gamma_k)+self.gamma_b
        self.beta = K.dot(self.conditioner,self.beta_k)+self.beta_b
        super(FiLM, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return x*self.gamma + self.beta

    def compute_output_shape(self, input_shape):
        return self.output_dim

In [10]:
l = FiLM((None,14,14,32))

In [ ]:
from keras.layers import Input
inputs = input_shape()
l()